In [2]:
# ==============================================================================
# Elastic Net
# ==============================================================================

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Path setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- 2) Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,    # expanding
    EWMA_CORR_SPEC,       # ewma (lambda=0.98 in config default)
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.EN import ForecastModel

# --- 3) Config ---
USE_DYNAMIC_FI_PIPELINE = True  # False = standard setup (I & II)
SEED = 42

MODEL_NAME = "elastic_net_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "elastic_net"

outputs_for_model(MODEL_NAME)
print(f"--- Starting tuning for: {MODEL_NAME} ---")

# --- 4) Load data ---
y = load_target()            # Target: ΔIP
X_ifo = load_ifo_features()  # Features: ifo (2160 columns in Track 1)

# Align indices
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

X_full_lagged = None
rolling_imp = None
y_fi = None

if USE_DYNAMIC_FI_PIPELINE:
    FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)

        # Intersection of all three indices
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print(f"Dynamic FI mode: loaded {X_full_lagged.shape[1]} features.")
    except FileNotFoundError:
        print("ERROR: Dynamic FI artifacts not found. Please run feature_importance.ipynb first.")
        sys.exit(1)
else:
    print(f"Full FE mode (Setup I/II): {X_ifo.shape[1]} base features.")

# --- 5) Config defaults (thesis policy) ---
def get_thesis_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window = 24
    cfg.policy_decay = 0.97
    cfg.selection_mode = "decayed_best"
    return cfg

cfg_obj = get_thesis_cfg()

# --- 6) Grid definition ---------------------------------------

def build_grid_full_fe():
    """Build the grid for Setup I (ifo) and Setup II (ifo + target blocks)."""
    # 1) Feature engineering options
    lag_candidates = [tuple(range(7))]  # fixed 0–6

    corr_opts = [
        {"corr_spec": dict(DEFAULT_CORR_SPEC)},
        {"corr_spec": dict(EWMA_CORR_SPEC)},
    ]

    k1_opts = [700]

    red_opts = [0.9, 1.0]

    dr_opts = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_kmax": 30, "pca_var_target": 0.99},
        {"dr_method": "pls", "pls_components": 30},
    ]

    # 2) Target blocks (setup variants)
    block_opts = [
        None,                         # Setup I
        # ["AR1", "Chronos"],          # Setup IIa
        # ["AR1", "Chronos", "TSFresh"]# Setup IIb
    ]

    # 3) Sample weighting
    weight_opts = [
        {"sample_weight_decay": None},
        {"sample_weight_decay": 0.99},
    ]

    # 4) Elastic Net hyperparameters
    l1_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]  # thesis alpha -> sklearn l1_ratio
    alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]  # thesis lambda -> sklearn alpha

    grid = []

    for lags, corr, k1, red, dr in product(lag_candidates, corr_opts, k1_opts, red_opts, dr_opts):
        base_fe = {
            "lag_candidates": lags,
            "k1_topk": k1,
            "redundancy_param": red,
            **dr,
            **corr,
        }

        for blocks, weights in product(block_opts, weight_opts):
            for l1, alpha in product(l1_ratios, alphas):
                hp = {
                    **base_fe,
                    "target_block_set": blocks,
                    **weights,
                    "alpha": l1,      # sklearn l1_ratio
                    "lambda": alpha,  # sklearn alpha
                    "seed": SEED,
                }
                grid.append(hp)

    return grid


def build_grid_dynamic_fi():
    """Grid for Setup III (dynamic feature importance). No FE tuning here; only model HPs."""
    n_features_list = [50]  # pragmatic choice for the grid

    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.99}]
    l1_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]
    alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]

    grid = []
    for n_feat, w, l1, alpha in product(n_features_list, weight_opts, l1_ratios, alphas):
        hp = {
            "n_features_to_use": n_feat,
            **w,
            "alpha": l1,
            "lambda": alpha,
            "seed": SEED,
        }
        grid.append(hp)
    return grid

# --- 7) Run ---------------------------------------------------

if USE_DYNAMIC_FI_PIPELINE:
    grid = build_grid_dynamic_fi()
    print(f"Dynamic FI grid size: {len(grid)} configurations.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
    )

else:
    grid = build_grid_full_fe()
    print(f"Full FE grid size (Setup I & II): {len(grid)} configurations.")
    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=10,
        min_survivors_per_block=0,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
    )

print("\nTuning finished.")



--- Starte Tuning für: elastic_net_dynamic_fi ---
INFO in load_ifo_features: Renaming columns to ensure validity.
Dynamic FI Modus: 4320 Features geladen.
Dynamic FI Grid Größe: 70 Konfigurationen.
[Stage A] Using DYNAMIC FI (Gleis 3) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=70
  - Config 1/70
    · Month 5/20 processed | running...RMSE=1.8377
    · Month 10/20 processed | running...RMSE=1.5652
    · Month 15/20 processed | running...RMSE=1.5172
    · Month 20/20 processed | running...RMSE=1.4371
  - Config 2/70
    · Month 5/20 processed | running...RMSE=1.8371
    · Month 10/20 processed | running...RMSE=1.5643
    · Month 15/20 processed | running...RMSE=1.5158
    · Month 20/20 processed | running...RMSE=1.4350
  - Config 3/70
    · Month 5/20 processed | running...RMSE=1.8319
    · Month 10/20 processed | running...RMSE=1.5558
    · Month 15/20 processed | running...RMSE=1.5039
    · Month 20/20 processed | running...RMSE=1.4171
  - Config 4/70
    · Month

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.233e+02, tolerance: 3.398e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.234e+02, tolerance: 3.398e-02 Linear regression models with null weight for the l1 regularization term are mor

    · Month 5/20 processed | running...RMSE=1.8359
    · Month 10/20 processed | running...RMSE=1.5616
    · Month 15/20 processed | running...RMSE=1.5134
    · Month 20/20 processed | running...RMSE=1.4323
  - Config 10/70
    · Month 5/20 processed | running...RMSE=1.8309
    · Month 10/20 processed | running...RMSE=1.5422
    · Month 15/20 processed | running...RMSE=1.4946
    · Month 20/20 processed | running...RMSE=1.4041
  - Config 11/70
    · Month 5/20 processed | running...RMSE=1.8008
    · Month 10/20 processed | running...RMSE=1.4626
    · Month 15/20 processed | running...RMSE=1.4721
    · Month 20/20 processed | running...RMSE=1.3708
  - Config 12/70
    · Month 5/20 processed | running...RMSE=1.6460
    · Month 10/20 processed | running...RMSE=1.3145
    · Month 15/20 processed | running...RMSE=1.2341
    · Month 20/20 processed | running...RMSE=1.1015
  - Config 13/70
    · Month 5/20 processed | running...RMSE=1.6460
    · Month 10/20 processed | running...RMSE=1.3145
 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.069e+02, tolerance: 3.275e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.070e+02, tolerance: 3.275e-02 Linear regression models with null weight for the l1 regularization term are mor

    · Month 20/20 processed | running...RMSE=1.0794
  - Config 48/70
    · Month 5/20 processed | running...RMSE=1.6219
    · Month 10/20 processed | running...RMSE=1.2897
    · Month 15/20 processed | running...RMSE=1.2131
    · Month 20/20 processed | running...RMSE=1.0794
  - Config 49/70
    · Month 5/20 processed | running...RMSE=1.6219
    · Month 10/20 processed | running...RMSE=1.2897
    · Month 15/20 processed | running...RMSE=1.2131
    · Month 20/20 processed | running...RMSE=1.0794
  - Config 50/70
    · Month 5/20 processed | running...RMSE=1.8469
    · Month 10/20 processed | running...RMSE=1.6499
    · Month 15/20 processed | running...RMSE=1.5427
    · Month 20/20 processed | running...RMSE=1.5440
  - Config 51/70
    · Month 5/20 processed | running...RMSE=1.8466
    · Month 10/20 processed | running...RMSE=1.6495
    · Month 15/20 processed | running...RMSE=1.5427
    · Month 20/20 processed | running...RMSE=1.5417
  - Config 52/70
    · Month 5/20 processed | runnin

...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.207e+03, tolerance: 2.434e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.199e+03, tolerance: 2.415e-01 Linear regression models with null weight for the l1 regularization term are mor

[Stage B] Month origin t=398 | evaluating 2 configs | active=2
[Stage B] Month origin t=399 | evaluating 2 configs | active=2
[Stage B] Month origin t=400 | evaluating 2 configs | active=2
[Stage B] Month origin t=401 | evaluating 2 configs | active=2
[Stage B] Month origin t=402 | evaluating 2 configs | active=2


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.179e+03, tolerance: 2.374e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.170e+03, tolerance: 2.356e-01 Linear regression models with null weight for the l1 regularization term are mor

[Stage B] Month origin t=403 | evaluating 2 configs | active=2
[Stage B] Month origin t=404 | evaluating 2 configs | active=2
[Stage B] Month origin t=405 | evaluating 2 configs | active=2
[Stage B] done → /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageB/elastic_net_dynamic_fi/monthly
[Stage B] summary.csv & summary_active.txt geschrieben.

Tuning abgeschlossen.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.188e+03, tolerance: 2.396e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.188e+03, tolerance: 2.396e-01 Linear regression models with null weight for the l1 regularization term are mor